# Stack Overflow Survey Data Preprocessing

This notebook performs data cleaning and preprocessing operations on the Stack Overflow Developer Survey dataset.

In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

## Define Data Paths

In [ ]:
# Define data paths
data_path = Path('../data')
interim_data_path = Path('../data/interim')
clean_v2_path = Path('../data/interim/clean_v2.csv')
output_path = Path('../data/interim/clean_v4.csv')
df = pd.read_csv(clean_v2_path)
print(f"Dataset loaded from {clean_v2_path} with {df.shape[0]} rows and {df.shape[1]} columns")

Dataset loaded from ..\data\interim\clean_v2.csv with 65437 rows and 15 columns


In [133]:
# Display basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ResponseId               65437 non-null  int64 
 1   MainBranch               65437 non-null  object
 2   Age                      65437 non-null  object
 3   Employment               65437 non-null  object
 4   EdLevel                  60784 non-null  object
 5   YearsCode                59869 non-null  object
 6   YearsCodePro             51610 non-null  object
 7   DevType                  59445 non-null  object
 8   LanguageHaveWorkedWith   59745 non-null  object
 9   DatabaseHaveWorkedWith   50254 non-null  object
 10  PlatformHaveWorkedWith   42366 non-null  object
 11  WebframeHaveWorkedWith   45161 non-null  object
 12  EmbeddedHaveWorkedWith   22214 non-null  object
 13  MiscTechHaveWorkedWith   39443 non-null  object
 14  ToolsTechHaveWorkedWith  52482 non-nul

## Data Cleaning

1. Drop specified columns
2. Filter out students
3. Remove rows with null values in DevType

In [ ]:
# Define columns to drop
columns_to_drop = ['ResponseId', 'MainBranch', 'Age', 'Employment','EdLevel','YearsCode','YearsCodePro']

# Check if columns exist in the dataframe
missing_columns = [col for col in columns_to_drop if col not in df.columns]
if missing_columns:
    print(f"Warning: The following columns are not in the dataframe: {missing_columns}")
    # Remove missing columns from columns_to_drop
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# Step 1: Drop specified columns
df_cleaned = df.drop(columns=columns_to_drop)
print(f"After dropping columns: {df_cleaned.shape[0]} rows and {df_cleaned.shape[1]} columns")

After dropping columns: 65437 rows and 11 columns


In [135]:
rows_before = df_cleaned.shape[0]

df_filtered = df_cleaned[
    (df_cleaned['DevType'] != 'Student') & 
    (~df_cleaned['DevType'].isna())
]

rows_after = df_filtered.shape[0]
rows_removed = rows_before - rows_after
percentage = (rows_removed / rows_before) * 100

print(f"Before filtering: {rows_before} rows")
print(f"After filtering: {rows_after} rows")
print(f"Removed {rows_removed} rows ({percentage:.2f}% of the dataset)")

Before filtering: 65437 rows
After filtering: 54343 rows
Removed 11094 rows (16.95% of the dataset)


In [136]:
# Check the new dataset
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54343 entries, 1 to 65435
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EdLevel                  54343 non-null  object
 1   YearsCode                54125 non-null  object
 2   YearsCodePro             49754 non-null  object
 3   DevType                  54343 non-null  object
 4   LanguageHaveWorkedWith   52495 non-null  object
 5   DatabaseHaveWorkedWith   45220 non-null  object
 6   PlatformHaveWorkedWith   38622 non-null  object
 7   WebframeHaveWorkedWith   40550 non-null  object
 8   EmbeddedHaveWorkedWith   18627 non-null  object
 9   MiscTechHaveWorkedWith   35246 non-null  object
 10  ToolsTechHaveWorkedWith  47116 non-null  object
dtypes: object(11)
memory usage: 5.0+ MB


## Save the Cleaned Dataset

## Additional Filtering

Remove rows where ALL technology-related fields are null:

In [137]:
# Define tech-related columns to check for nulls
tech_columns = [
    'LanguageHaveWorkedWith',
    'DatabaseHaveWorkedWith',
    'PlatformHaveWorkedWith',
    'WebframeHaveWorkedWith',
    'EmbeddedHaveWorkedWith',
    'MiscTechHaveWorkedWith',
    'ToolsTechHaveWorkedWith'
]

# Number of rows before tech field filtering
rows_before_tech_filter = df_filtered.shape[0]

# Filter out rows where ALL technology columns are null
df_filtered = df_filtered[~df_filtered[tech_columns].isna().all(axis=1)]

# Number of rows after tech field filtering
rows_after_tech_filter = df_filtered.shape[0]
rows_removed_tech_filter = rows_before_tech_filter - rows_after_tech_filter
percentage_tech_filter = (rows_removed_tech_filter / rows_before_tech_filter) * 100 if rows_before_tech_filter > 0 else 0

print(f"Before tech field filtering: {rows_before_tech_filter} rows")
print(f"After tech field filtering: {rows_after_tech_filter} rows")
print(f"Removed {rows_removed_tech_filter} rows ({percentage_tech_filter:.2f}% of the remaining dataset)")

Before tech field filtering: 54343 rows
After tech field filtering: 52692 rows
Removed 1651 rows (3.04% of the remaining dataset)


## Filter Specific DevType Values

Remove rows with specific DevType values that are not of interest for our analysis:

In [ ]:
# Define DevType values to drop
dev_types_to_drop = [
    'Scientist',
    'Developer Advocate',
    'Research & Development role',
    'Developer Experience',
    'Designer',
    'Senior Executive (C-Suite, VP, etc.)',
    'Educator',
    'Developer, QA or test',
    'Engineer, site reliability',
    'Marketing or sales professional',
    'Other (please specify):',
    'Academic researcher',
    'Engineering manager',
    ' Product manager',
    ' Project manager',
    ' Security professional '
]

# Count rows before DevType filtering
rows_before_devtype_filter = df_filtered.shape[0]

# Filter out rows with specified DevType values
df_filtered = df_filtered[~df_filtered['DevType'].isin(dev_types_to_drop)]

# Count rows after DevType filtering
rows_after_devtype_filter = df_filtered.shape[0]
rows_removed_devtype = rows_before_devtype_filter - rows_after_devtype_filter
percentage_devtype = (rows_removed_devtype / rows_before_devtype_filter) * 100 if rows_before_devtype_filter > 0 else 0

print(f"Before DevType filtering: {rows_before_devtype_filter} rows")
print(f"After DevType filtering: {rows_after_devtype_filter} rows")
print(f"Removed {rows_removed_devtype} rows ({percentage_devtype:.2f}% of the remaining dataset)")

Before DevType filtering: 52692 rows
After DevType filtering: 45377 rows
Removed 7315 rows (13.88% of the remaining dataset)


In [139]:
# Display unique values in the DevType column
unique_dev_types = df_filtered['DevType'].dropna().unique()
for dev_type in unique_dev_types:
    count = df[df['DevType'].str.contains(dev_type, na=False)].shape[0]
    print(f"{dev_type}: {count} rows")

Developer, full-stack: 18260 rows
Project manager: 418 rows
Developer, back-end: 9928 rows
Developer, front-end: 3349 rows
Database administrator: 171 rows
Developer, desktop or enterprise applications: 2493 rows
Cloud infrastructure engineer: 634 rows
Data scientist or machine learning specialist: 1024 rows
Developer, embedded applications or devices: 1623 rows
System administrator: 552 rows
DevOps specialist: 1019 rows
Engineering manager: 1275 rows
Security professional: 356 rows
Developer, mobile: 2021 rows
Developer, game or graphics: 706 rows
Data or business analyst: 523 rows
Product manager: 290 rows
Developer, AI: 543 rows
Blockchain: 235 rows
Hardware Engineer: 200 rows
Data engineer: 1118 rows


In [ ]:
# Define DevType values to drop
dev_types_to_drop = [
    'Scientist',
    'Developer Advocate',
    'Research & Development role',
    'Developer Experience',
    'Designer',
    'Senior Executive (C-Suite, VP, etc.)',
    'Educator',
    'Developer, QA or test',
    'Engineer, site reliability',
    'Marketing or sales professional',
    'Other (please specify):',
    'Academic researcher',
    'Engineering manager',
    'Product manager',
    'Project manager',
    'Security professional',
    'Hardware Engineer'
]

# Count rows before DevType filtering
rows_before_devtype_filter = df_filtered.shape[0]

# Filter out rows with specified DevType values
df_filtered = df_filtered[~df_filtered['DevType'].isin(dev_types_to_drop)]

# Count rows after DevType filtering
rows_after_devtype_filter = df_filtered.shape[0]
rows_removed_devtype = rows_before_devtype_filter - rows_after_devtype_filter
percentage_devtype = (rows_removed_devtype / rows_before_devtype_filter) * 100 if rows_before_devtype_filter > 0 else 0

print(f"Before DevType filtering: {rows_before_devtype_filter} rows")
print(f"After DevType filtering: {rows_after_devtype_filter} rows")
print(f"Removed {rows_removed_devtype} rows ({percentage_devtype:.2f}% of the remaining dataset)")

Before DevType filtering: 44129 rows
After DevType filtering: 43103 rows
Removed 1026 rows (2.33% of the remaining dataset)


In [3]:
# Save the cleaned dataset to clean_v3.csv
df_filtered.to_csv(output_path, index=False)
print(f"Dataset saved to {output_path} with {df_filtered.shape[0]} rows and {df_filtered.shape[1]} columns")

Dataset saved to ../../data/processed/clean_v3.csv with 42909 rows and 8 columns


## Summary of Changes

1. Dropped columns: ResponseId, MainBranch, Age, Employment
2. Removed entries with DevType = 'Student' 
3. Removed entries with null values in the DevType column
4. Removed entries where ALL technology-related columns are null
5. Removed entries with specific DevType values: Scientist, Developer Advocate, Research & Development role, Developer Experience, Designer, Senior Executive, Educator, (Developer, QA or test), (Engineer, site reliability), Marketing or sales professional, Other
6. Saved the cleaned dataset to clean_v5.csv